## Coletando dados de zonas e páginas do mercado livre e salvando em um dataframe

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [ ]:
zonas = ['sul', 'norte'] #, 'leste', 'oeste']
paginas = ['', '_Desde_49'] #, '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']

In [ ]:
url_ml = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}' 

In [ ]:
def funcao_zona(zona):
    lista_zonas = []
    lista_zonas.append(zona)

In [ ]:
for zona in zonas:
    for pagina in paginas:
        funcao_zona(zona)
        print(pagina, zona)

 sul
_Desde_49 sul
 norte
_Desde_49 norte


In [ ]:
re_quarto ='\| (.*) quarto'
re_area = '> (.*) m²'

In [ ]:
def coleta_dados(url, zona):

    conteudo = requests.get(url).content
    sopa = BeautifulSoup(conteudo)
    
    precos = sopa.find_all('span', class_='price__fraction')
    areas_e_quartos = sopa.find_all('div', class_='item__attrs')
    
    lista_areas = []
    lista_quartos = []
    lista_precos = []
    lista_zonas = []
    
    for padrao in precos:
        preco = re.findall('<span class="price__fraction">(.*)</span>', str(padrao))
        lista_precos.append(preco[0].replace('.', ''))
    
    for elemento in areas_e_quartos:
      quarto = re.findall(re_quarto, str(elemento))
      area = re.findall(re_area, str(elemento))
      
      if quarto==[]:
          quarto=np.nan
      else:
          quarto=quarto[0]
        
      if area==[]:
          area=np.nan
      else:
          area=area[0]

      lista_quartos.append(quarto)
      lista_areas.append(area)
      lista_zonas.append(zona)

    dados = {'zona': lista_zonas,
           'quartos': lista_quartos,
           'area': lista_areas,
           'preco': lista_precos}

    df = pd.DataFrame(dados)  
    return df

In [ ]:
lista_dfs = []
for zona in zonas:
    for pagina in paginas:
        url = url_ml.format(zona, pagina)
        df = coleta_dados(url, zona)
        print('Carregando a url', url)
        lista_dfs.append(df)
        sleep(2)
print('Dados coletados!')

Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando a url https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Dados coletados!


In [ ]:
dados = pd.concat(lista_dfs)

In [ ]:
dados

,zona,quartos,area,preco
0,sul,1,30,700
1,sul,2,90,1700
2,sul,1,30,600
3,sul,1,40,850
4,sul,1,20,500
...,...,...,...,...
43,norte,1,35,700
44,norte,2,90,2200
45,norte,1,45,1050
46,norte,3,250,2100


In [ ]:
dados.to_csv('dados_mercado_livre.csv', index=False)

In [ ]:
df = pd.read_csv('dados_mercado_livre.csv')

In [ ]:
type(dados)

pandas.core.frame.DataFrame

In [ ]:
# pd.DataFrame.to_csv?

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   zona     192 non-null    object 
 1   quartos  167 non-null    float64
 2   area     186 non-null    float64
 3   preco    192 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 6.1+ KB


In [ ]:
df.count()

zona       192
quartos    167
area       186
preco      192
dtype: int64

In [ ]:
df.isnull().sum()

zona        0
quartos    25
area        6
preco       0
dtype: int64

In [ ]:
df.shape # linhas, colunas

(192, 4)